In [9]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'rids-first-project'

In [10]:
REGION = 'us-west1'

### Packages

In [26]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets
import numpy as np
import matplotlib as plt
import json

### Clients

In [12]:
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

### Parameters

In [13]:
BUCKET = PROJECT_ID

## Create Storage Bucket

In [14]:
# Check to see if bucket already exist and create if missing
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Bucket already exist: rids-first-project


In [15]:
print(f'Review the storage bucket in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

Review the storage bucket in the console here:
https://console.cloud.google.com/storage/browser/rids-first-project;tab=objects&project=rids-first-project


## Service Account & Permissions

This notebook instance is running as a service account in GCP. This service account will also be used to run other services in Vertex AI like training jobs and pipelines. The service account will need permission to interact with object in Cloud Storage which requires the role (roles/storage.objectAdmin).

Get the current service account:

In [16]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'97651066104-compute@developer.gserviceaccount.com'

### Enable the Cloud Resource Manager API:

In [17]:
!gcloud services enable cloudresourcemanager.googleapis.com

### List the service accounts current roles:

In [18]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/storage.objectAdmin


#### If the resulting list is missing roles/storage.objectAdmin or another role that contains this permission, like the basic role roles/owner, then it will need to be added for the service account. Use these instructions to complete this:

In [16]:
print(f'Go To IAM in the Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')

Go To IAM in the Google Cloud Console:
https://console.cloud.google.com/iam-admin/iam?orgonly=true&project=rids-first-project&supportedpurview=organizationId


Go To IAM in the Google Cloud Console:
https://console.cloud.google.com/iam-admin/iam?orgonly=true&project=statmike-mlops-349915&supportedpurview=organizationId

From the console link above, or by going to https:/console.cloud.google.com and navigating to "IAM & Admin > IAM":

1. Locate the row for the service account listed above: -compute@developer.gserviceaccount.com
2. Under the inheritance column click the pencil icon to edit roles
3. In the fly over menu, under Assign roles select Add Another Role
4. Click the Select a role box and type Storage Object Admin, then select Storage Object Admin
5. Click Save
6. Rerun the list of services below and verify the role has been added:

In [20]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/bigquery.jobUser
roles/owner
roles/storage.objectAdmin


# Install Kubeflow Pipelines (KFP)

In [31]:
!pip install kfp -U -q

ERROR: Cannot uninstall jsonschema 4.17.3, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps jsonschema==4.17.3'.


In [4]:
!pip install google-cloud-pipeline-components -U -q

ERROR: Cannot uninstall jsonschema 4.17.3, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps jsonschema==4.17.3'.


In [35]:
pip install --force-reinstall --no-deps jsonschema==4.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.17.3
ERROR: Cannot uninstall jsonschema 4.17.3, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps jsonschema==4.17.3'.
Note: you may need to restart the kernel to use updated packages.


## Update AIPlatform Package:

The google-cloud-aiplatform package updates frequently. Update it for latest functionality.

aiplatform Python Client
GitHub Repo for api-common-protos
For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here aiplatform_notes.md.

In [7]:
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q